# Lab 8: Grid Localization using Bayes Filter

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


# Python Classes
There are five major classes (VirtualRobot, Mapper, BaseLocalization, Plotter and Trajectory)  defined in **robot_interface.py** that provide you with the necessary functions and variables for working your way through the lab. 
In this notebook, we discuss only the relevant functions and variables that are of use for this lab. 
You may refer to the python file for an understanding of how the various functions work, however, this is not entirely necessary as the Jupyter notebook should provide you with all the necessary information.
<hr>

## VirtualRobot
The class provides member functions required to interact with the virtual robot in the simulator. <br> 

**Make sure the simulator is running.** <br>

### Member Functions:
- <span style="color:rgb(255,191,0)">**get_pose()**</span>:  Returns the latest odometry robot pose estimate as a tuple of the format $(x, y, yaw)$ with units $(meters, meters, degrees)$.

- <span style="color:rgb(255,191,0)">**get_gt_pose()**</span>: Returns the latest ground truth robot pose as a tuple of the format $(x, y, yaw)$ with units $(meters, meters, degrees)$.


**<ins>NOTE</ins>**: The above functions provide angle information in degrees.

<hr>

## Mapper
The class provides member functions to handle various operations related to the grid map. <br>
The discrete grid map spans a world of $(-2, 2)~meters$ in the x-axis, $(-2,2)~meters$ in the y-axis and $[-180, 180)~degrees$ in the $\theta$-axis. <br>
The grid cell size in the x, y, and angle axes are 0.2 m, 0.2 m and 20 deg, respectively. <br>
The grid has $(20, 20, 18)$ cells along the different dimensions, with a total number of 7200 cells. <br>   

### Member Functions:
- <span style="color:rgb(255,191,0)">**from_map(cx, cy, ca)**</span>: Returns the continuous world coordinates *(x,y,a)* of the center of the grid cell index *(cx, cy, ca)*. 
- <span style="color:rgb(255,191,0)">**to_map(x, y, a)**</span>: Returns the grid map cell index *(cx, cy, ca)* of the continuous world coordinates *(x,y,a)*.
- <span style="color:rgb(255,191,0)">**normalize_angle(a)**</span>: Normalizes the angle *a* to the interval \[-180, 180).
- <span style="color:rgb(255,191,0)">**get_views(cx, cy, ca)**</span>: Returns a 1D array of the precached true measurements (views) at a given grid cell index *(cx, cy, ca)*. The true measurements are calculated offline by performing ray cast operations from the center of each cell in the map.

### Member Variables:
- <span style="color:rgb(100,191,100)">**MAX_CELLS_X**</span>: Maximum number of grid cells in the X direction.
- <span style="color:rgb(100,191,100)">**MAX_CELLS_Y**</span>: Maximum number of grid cells in the Y direction.
- <span style="color:rgb(100,191,100)">**MAX_CELLS_A**</span>: Maximum number of grid cells in the $\theta$ direction ('A' stands for 'Angle').
- <span style="color:rgb(100,191,100)">**OBS_PER_CELL**</span>: Number of observations per cell. This is set to 18.
- <span style="color:rgb(100,191,100)">**obs_views**</span>: A 4D Numpy array where the first three dimensions represents the cell index (cx, cy, ca) and the fourth dimension represents the precached true measurements (views). Hence the size of the Numpy array is (20,20,18,18). You may think of it as a 3D array representing the grid where each cell stores 18 precached true measurements calculated using ray casting from the center of that cell.

**<ins>NOTE</ins>**: **Do not change the map properties** such as discretization size, map size, maximum number of cells, etc as they are tuned for the specific lab setting.
<hr>

In [94]:
# Test block
obs = loc.obs_range_data
sensor_sigma = loc.sensor_sigma
cur_pose = mapper.from_map(10,10,9)
#for i in range(18):  # one reading for each value of theta 
#    prob_array[:,:,i] = gaussian(obs[i], mapper.obs_views[:,:,i,i],sensor_sigma)
#print("Before: {}".format(sum(sum(sum(prob_array)))))
#prob_array = prob_array / sum(sum(sum(prob_array)))   # normalized probability
#print("After: {}".format(sum(sum(sum(prob_array)))))
#print(mapper.get_views(10,10,9))
#print(mapper.get_views(10,10,10))
# Ohhh. So there are 18 views corresponding to absolute values of theta,
# and then they are "rotated" by the value of theta you give the function.
#for i in range(18):  # one reading for each value of theta
    #    prob_array[:,:,i] = gaussian(obs[i], mapper.obs_views[:,:,i,0],sensor_sigma)
theta = cur_pose[2] # what angle were we facing when we started?
thIndex = to_map(1,1,normalize_angle(theta))[2]
prob_array[:,:,:] = gaussian(obs[i], mapper.obs_views[:,:,thIndex,:],sensor_sigma)
prob_array.shape

(20, 20, 18)

## Plotter
The Plotter class provides various member functions to plot and visualize data. <br>

### Member Functions:

- <span style="color:rgb(255,191,0)">**plot_point(x, y, plot_type)**</span>: Plots a point at the position *x*,*y*. The color of the plotted point depends on *plot_type* which can be equal to one of the following: *ODOM, GT, or BEL* for blue, green or yellow, respectively.

- <span style="color:rgb(255,191,0)">**visualize_map()**</span>: Visualizes the map in the plotter as white lines segments as defined in the member variable **lines** of class **Mapper**.

- <span style="color:rgb(255,191,0)">**visualize_prior_bel()**</span>: Visualizes the prior belief(i.e belief after the prediction step) in the plotter as a grid map where each cell has a grayscale value depending on the probability of the cell; darkers regions indicate lower probability. **Make sure the "Dist." toggle is switched on in the plotter**.

- <span style="color:rgb(255,191,0)">**visualize_bel()**</span>: Visualizes the belief(i.e belief after the update step) in the plotter as a grid map where each cell has a grayscale value depending on the probability of the cell; darkers regions indicate lower probability. **Make sure the "Dist." toggle is switched on in the plotter**.

- <span style="color:rgb(255,191,0)">**reset_plot()**</span>: Resets the plots in the plotter. This removes all the previous plotted information.

- <span style="color:rgb(255,191,0)">**reset_map()**</span>: Clears the map visualization in the plotter.


**<ins>NOTE</ins>**: The functions *visualize_prior_bel()* and *visualize_bel()* visualize a 3D (grid) Belief in a 2D (grid) space by marginalizing over  $\theta$ i.e $bel(x,y) = \sum_{\theta} bel(x,y,\theta)$ and $\overline{bel}(x,y) = \sum_{\theta} \overline{bel}(x,y,\theta)$. <br>
<hr>

## BaseLocalization
The class provides member functions and member variables required for Grid Localization. <br>
Below is a quick summary of the necessary member functions and variables that will be required to complete this lab:

### Member Functions:
- <span style="color:rgb(255,191,0)">**init_pose(x,y,a)**</span>: Initialize the member variable **bel** with a point mass distribution centered at *(x,y,a)*.
- <span style="color:rgb(255,191,0)">**gaussian(x, mu, sigma)**</span>: Returns the relative likelihood of *x* in a Normal Distribution with mean *mu* and standard deviation *sigma*.
- <span style="color:rgb(255,191,0)">**get_observation_data()**</span>: Executes a rotation behavior where the robot rotates in place and collects 18 (=*OBS_PER_CELL*)  range measurements at equidistant angular intervals over a 360 degree space. This mimics a 360 degree laser scanner. The range measurements are stored in the member variable **obs_range_data** of class **BaseLocalization**.
- <span style="color:rgb(255,191,0)">**print_prediction_stats(plot_data)**</span>: A helper function that displays statistics after your prediction step and plots the ground truth position (green), odometry position (blue) and visualizes the prior belief (grayscale grid) if *plot_data* is True.
- <span style="color:rgb(255,191,0)">**print_update_stats(plot_data)**</span>: A helper function that displays statistics after your update step and plots the belief position (yellow) if *plot_data* is True.

### Member Variables:
- <span style="color:rgb(100,191,100)">**obs_range_data**</span>: A 1D Numpy array containing range measurements made by a robot after completing a rotation behaviour i.e after calling the function **get_observation_data()**.
- <span style="color:rgb(100,191,100)">**bel_bar**</span>: A 3D Numpy array that is used to represent the prior belief of the robot i.e after the most recent prediction step.  
- <span style="color:rgb(100,191,100)">**bel**</span>: A 3D Numpy array that is used to represent the belief of the robot i.e after the most recent update step. It is initialized with a point mass distribution centered at $(0~m,0~m,0~deg)$ or grid index $(10,10,9)$.
- <span style="color:rgb(100,191,100)">**sensor_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for your sensor model.
- <span style="color:rgb(100,191,100)">**odom_rot_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for rotation in your odometry motion model.
- <span style="color:rgb(100,191,100)">**odom_trans_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for translation in your odometry motion model.

**<ins>NOTE</ins>**: The noise parameters are dependent on the robot motion model, sensor model and grid discretization. The values are thus tuned to work for the lab setting. **Do not change the noise parameters.**
<hr>

## Trajectory
The trajectory class encodes a pre-planned collision free trajectory to be executed by your robot. The class is defined in **Traj.py**. <br>

### Member Functions:
<span style="color:rgb(255,191,0)">**execute_time_step(t)**</span>: Executes the motion at time step *t* and returns *prev_odom, current_odom, prev_gt, current_gt*.
   - prev_odom: odometry pose before executing the motion.    
   - current_odom: odometry pose after executing the motion.
   - prev_gt     : ground truth pose before executing the motion.
   - current_gt  : ground truth pose after executing the motion.

### Member Variables:
<span style="color:rgb(100,191,100)">**total_time_steps**</span>: The total number of time steps in the trajectory (26). 


Thus, calling the member function **execute_time_step(t)** repeatedly from t=0 to t=total_time_steps-1 will make the robot execute the entire collision-free, pre-planned trajectory.
## <span style="color:rgb(240,50,50)"> Do not use the mouse or the keyboard teleop tool to move the robot while executing the pre-planned trajectory motion. </span>
<hr>

In [61]:
# Automatically reload changes in python modules
%load_ext autoreload
%autoreload 2

# Import classes
from robot_interface import *

import time
import numpy as np
import rospy
from Traj import Trajectory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# Initialize Robot to communicate with the virtual/real robot and plotter
robot = VirtualRobot()

# Initialize mapper
# Requires a VirtualRobot object as input
mapper = init_map(robot)

# Initialize your localization object
# Requires a robot VirtualRobot object and a Mapper object as inputs
loc = BaseLocalization(robot, mapper)

# Visualize the map (described using line segments) in the plotter
loc.plotter.visualize_map()

Using python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0] 

Initializing Node
Initializing Virtual Robot
Initializing Mapper
 | Number of observations per grid cell:  18
 | Precaching Views...
 | Precaching Time:  44.658037424087524
Initial Pose:  (10, 10, 9)
Initializing belief with a Point mass Distribution at:  (10, 10, 9)


In [59]:
# Test block
from numpy import array, arange, hypot, arctan2, deg2rad, rad2deg
odom_rot_sigma = loc.odom_rot_sigma
odom_trans_sigma = loc.odom_trans_sigma
gaussian = loc.gaussian


tic = time.clock_gettime(time.CLOCK_MONOTONIC)
bel = np.ones([20,20,18])  # Test with uniform distrib.
bel[10,10,9] = 1e6         # Test with center point much more likely
bel = bel / sum(sum(sum(bel)))

prev_pose = [0.1,0.1,0]
cur_pose = [0.1,1.9,0]
u = compute_control(cur_pose,prev_pose)
drot1 = np.empty([20,20,1])   # first rotation to travel to point x,y
dtrans = np.empty([20,20,1])  # translation to travel to x,y after rotating
drot2 = np.empty([20,20,18])  # second rotation to achieve angle theta
(x,y,th) = mapper.to_map(*cur_pose) # actually the indices for x,y,th
(x0,y0,th0) = mapper.to_map(*prev_pose)
# Use vectorized code to calculate prevX & prevY fast (prevTh is a dummy for now)
(prevX,prevY,prevTh) = mapper.from_map(arange(0,20),arange(0,20),1)
dx = cur_pose[0]-prevX # change in x position (m) - array size 20
dy = cur_pose[1]-prevY # change in y position (m) - array size 20
# ^ Just do this once. th has to be recalculated many times because
# from_map() can't handle multiple values of theta at once.
for i in range(20):
    dtrans[i,:,0] = hypot(dx[i],dy)   # do all of y at once; loop through x
    drot1[i,:,0] = rad2deg(arctan2(dy,dx[i]))   # both 20x20 arrays
print("To reach position")
print("x={},y={},th={}".format(*cur_pose))
print("with indices a={},b={},c={}".format(x,y,th))
print("from position")
print("x={},y={},th={}".format(*prev_pose))
print("with indices a={},b={},c={}".format(x0,y0,th0))
print("dx = {}; dy = {}".format(dx[x0],dy[y0]))
print(drot1[x0,y0,0])
print(dtrans[x0,y0,0])
for i in range(18): # loop over all angles and overwrite th
    prevTh = mapper.from_map(1,1,i)[2]  # don't care about 0 and 1
    # Normalize the change in angle (could be >180°)
    dth = normalize_angle(cur_pose[2] - prevTh)
    drot2[:,:,i] = dth - drot1[:,:,0]  # build the entire array!
print(drot2[x0,y0,th0])
prob = sum(sum(sum(
          gaussian(u[0], drot1, odom_rot_sigma)
        * gaussian(u[1], dtrans, odom_trans_sigma)
        * gaussian(u[2], drot2, odom_rot_sigma) 
        * bel )))
print(prob)
toc = time.clock_gettime(time.CLOCK_MONOTONIC)
print("It takes {:.3} s to run a single iteration of the motion model".format(toc-tic))
# th = np.zeros([18])
# dth = th
# for i in range(18):
#     (x,y,th[i]) = mapper.from_map(arange(0,20),arange(0,20),i)
#     dth[i] = mapper.normalize_angle(th[i] - prev_pose[2])
# dx = x-prev_pose[0] # change in x position (m) - array size 20
# dy = y-prev_pose[1] # change in y position (m) - array size 20
# # Normalize the change in angle (could be >180°)
# dtrans = hypot(dx,dy)   # get magnitude of dx, dy 
# drot1 = rad2deg(arctan2(dy,dx))
# drot2 = dth - drot1[0:18] # these two arrays are different sizes.
# # what I really want to do is subtract every element of drot1 from dth separately and get 18 arrays,
# # not truncate it and get a weird set.
# # So this needs to be in a for loop.
# # Probability of being in this pose given the control action taken:
# prob = (gaussian(u[0], drot1[0:18], odom_rot_sigma) *
#         gaussian(u[1], dtrans[0:18], odom_trans_sigma) *
#         gaussian(u[2], drot2, odom_rot_sigma) )
# #print(prob)
# poses = np.zeros([20,20,1,3])
# for i in range(20):
#     for j in range(20):
#         poses[i,j,0,:] = mapper.from_map(i,j,9)
#print(poses[0,0,0,2])

To reach position
x=0.1,y=1.9,th=0
with indices a=10,b=19,c=9
from position
x=0.1,y=0.1,th=0
with indices a=10,b=10,c=9
dx = 0.0; dy = 1.7999999999999998
90.0
1.7999999999999998
-100.0
0.7950588935185371
It takes 0.0105 s to run a single iteration of the motion model


# Execute Trajectory
The below cell showcases how to use the **Trajectory** class to go through the pre-planned motion trajectory. <br>
The cell does not run any sort of probabilistic estimation. It simply executes the motion in each time step in a successive manner. 

Run this cell once and monitor how the robot executes the entire, collision-free trajectory.

**Make sure you go through the motion in a successive manner in order to execute a collision free trajectory.** 

In [63]:
loc.plotter.reset_plot()

# Intialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    # Execute control at t=0
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
    
    # Plot Odom and GT
    loc.plotter.plot_point(current_odom[0], current_odom[1], ODOM)
    loc.plotter.plot_point(current_gt[0], current_gt[1], GT)
    


 | Resetting Robot pose


# Get Observation Data

The robot is designed to executes a 360 rotation behavior where it rotates in place to collect 18 (=*OBS_PER_CELL*) range measurements at equidistant angular intervals over the 360 degree space. This mimics a 360 degree laser scanner. The range measurements are stored in the member variable **obs_range_data** of class **BaseLocalization**.

The cell below runs the **get_observation_data()** to execute this pre-planned behavior once at its current pose. Notice the virtual robot rotating in-place in the simulator.

In [64]:
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Print the latest observation data stored in the member variable obs_range_data
print(loc.obs_range_data)

 | Executing Observation Loop at: 30 deg/s
[0.75214678 2.65370154 0.75716662 0.87632143 1.93074369 1.79092407
 1.6636101  2.14348674 1.83463061 1.91877139 2.20458722 1.26001716
 1.03628588 1.12761927 2.39320993 2.98438191 2.25225258 0.98424667]


The above behavior will be repeated after each time step in the pre-planned trajectory to get the required observation data ($z_{t}$) at each time step.

<hr>

# Localization
<span style="color:rgb(0,150,0)">Make sure you refer to the section "Implementation Tips" in the lab documentation.</span> <br>

The cell below contains skeleton code for various functions required to perform grid localization using Bayes Filter. <br> 
Write necessary code for each function definition. <br>
You may add more functions or edit a function's definition to have a different input (arguments) and/or output (return values). <br>
For example, you may use numpy matrix operations to calculate the sensor model directly in the **update_step()** function and thus skip the *sensor_model()* function.<br>

### <span style="color:rgb(0,150,0)">If you make changes to any of the functions below, make sure to re-run the below cell.</span> <br>


In [65]:
# Import useful Numpy functions
from numpy import arctan2, hypot, pi, deg2rad, rad2deg, arange
# and alias several functions to prevent errors later
odom_rot_sigma = loc.odom_rot_sigma
odom_trans_sigma = loc.odom_trans_sigma
sensor_sigma = loc.sensor_sigma
gaussian = loc.gaussian
normalize_angle = mapper.normalize_angle
to_map = mapper.to_map
toMap = to_map
from_map = mapper.from_map
fromMap = from_map

# In world coordinates
def compute_control(cur_pose, prev_pose):
    """ Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    Pseudocode (well, this code probably compiles):
        delta_y = cur_pose[1] - prev_pose[1]
        delta_x = cur_pose[0] - prev_pose[0]
        delta_rot_1 = atan2(delta_y, delta_x)
        delta_trans = sqrt(delta_y^2 + delta_y^2)
        delta_rot_2 = cur_pose[2] - (prev_pose[2] + delta_rot_1)
    """
    dy = cur_pose[1] - prev_pose[1] # pre-compute these since they'll be
    dx = cur_pose[0] - prev_pose[0] # used twice
    # normalize the change in angle (could be >180°)
    dTheta = normalize_angle(cur_pose[2] - prev_pose[2])
    delta_rot_1 = rad2deg(arctan2(dy, dx))
    delta_trans = hypot(dx, dy) # get magnitude of dx, dy
    delta_rot_2 = dTheta - delta_rot_1
    return delta_rot_1, delta_trans, delta_rot_2

# In world coordinates
def odom_motion_model(cur_pose, bel, u):
    """ Odometry Motion Model

    Args:
        cur_pose  ([Pose]): Current Pose (x,y,th) in meters
        bel ([ndarray 20x20x18]): Belief about position from last iteration
        u = (rot1, trans, rot2) (float, float, float): A tuple with control data
            in the format (rot1, trans, rot2) with units (degrees, meters, degrees)

    Returns:
        prob [scalar float]: Probability sum over x_{t-1} (p(x'|x, u)) at the cur_pose.
        
    Pseudocode (MODIFIED):
        x, y, th = prev_pose
        for prevX,prevY,prevTh in gridPoints: # use fromMap() to get these coords.
            # Figure out what each movement would have been to get here
            dx = x-prevX; dy = y-prevY; dth = theta-prevTh;
            dtrans = sqrt(dx^2+dy^2)
            drot1 = atan2(dy,dx)
            drot2 = dth - rot1i
            pR = gaussian(trans, dtrans, odom_trans_sigma)
            pTh1 = gaussian(rot1, drot1, odom_rot_sigma)
            pTh2 = gaussian(rot2, drot2, odom_rot_sigma)
            pXYT = pR*pTh1*pTh2 # probability we got all three right
            prob[x,y,th] = prob[x,y,th] + pXYT*bel[prevX,prevY,prevTh]
    """
    # Determine what each movement would have been to get here: 20x20x18 array
    # We don't know where we are or where we were, but we have
    # * A guess for where we are now (cur_pose).
    # * A distribution for where we were. Sum over that in this function.
    drot1 = np.empty([20,20,1])   # first rotation to travel to point x,y
    dtrans = np.empty([20,20,1])  # translation to travel to x,y after rotating
    drot2 = np.empty([20,20,18])  # second rotation to achieve angle theta
    (x,y,th) = mapper.to_map(*cur_pose) # actually the indices for x,y,th
    # Use vectorized code to calculate prevX & prevY fast (prevTh is a dummy for now)
    (prevX,prevY,prevTh) = mapper.from_map(arange(0,20),arange(0,20),1)
    dx = cur_pose[0]-prevX # change in x position (m) - array size 20
    dy = cur_pose[1]-prevY # change in y position (m) - array size 20
    # ^ Just do this once. th has to be recalculated many times because
    # from_map() can't handle multiple values of theta at once.
    for i in range(20):
        dtrans[i,:,0] = hypot(dx[i],dy)   # do all of y at once; loop through x
        drot1[i,:,0] = rad2deg(arctan2(dy,dx[i]))   # both 20x20 arrays
    for i in range(18): # loop over all angles and overwrite th
        prevTh = mapper.from_map(1,1,i)[2]  # don't care about 0 and 1
        # Normalize the change in angle (could be >180°)
        dth = normalize_angle(cur_pose[2] - prevTh)
        drot2[:,:,i] = dth - drot1[:,:,0]  # build the entire array!
    # Probability of being in this pose given the control action taken:
    # use Law of Total Probability and sum over all possible starting points.
    # This is the entire RHS of the pseudocode for the prediction step.
    prob = sum(sum(sum(
              gaussian(u[0], drot1, odom_rot_sigma)
            * gaussian(u[1], dtrans, odom_trans_sigma)
            * gaussian(u[2], drot2, odom_rot_sigma) 
            * bel )))
    return prob

def prediction_step(cur_odom, prev_odom, bel):
    """ Prediction step of the Bayes Filter.
    Update the probabilities in loc.bel_bar based on loc.bel (bel)
    from the previous time step and the odometry motion model.

    Args:
        cur_odom  ([Pose]): Current pose, estimated from odometry
        prev_odom ([Pose]): Previous pose, estimated from odometry
        loc.bel [ndarray 20x20x18]: Previous location probability density
        
    Returns:
        loc.bel_bar [ndarray 20x20x18]: Updated location probability density.
        This will be the prior belief for the update step.
        
    Pseudocode (FIXED):
        for x, y, theta in configuration space:
            loc.bel_bar[x,y,theta] = odom_motion_model((x,y,th),bel,u)
            # Plugs in all possible current values (t).
            # odom_motion_model() takes care of
            # summing over all possible previous values (t-1).
    """
    u = compute_control(cur_odom,prev_odom) # What control action did we take?
    bel_bar = np.zeros([20,20,18]) # Initialize prior belief with correct dims
    for k in range(18):     # loop over all possible current poses,
        for j in range(20): # in configuration space
            for i in range(20):
                pose = mapper.from_map(i,j,k) # from config space to world
                # odom_motion_model sums probability of getting here from
                # all possible previous poses
                bel_bar[i,j,k] = odom_motion_model(pose,bel,u)
                # so bel_bar is the probability of being at indices i,j,k
    # Below: just for debugging
#     (i,j,k) = (10,10,9)
#     print("bel_bar:")
#     print(bel_bar[i,j,k])
#     print(bel_bar.shape)
    return bel_bar

def sensor_model(obs):
    """ This is the equivalent of p(z|x). Checks all possible poses x_t.
    Args:
        obs [ndarray]: A 1D array consisting of the measurements made in rotation loop
                            1D array of size 18 (=loc.OBS_PER_CELL)

    Returns:
        probArray [ndarray]: Returns a 20x20x18 array corresponding to
                             p(z_t|x_t) for all values of x_t.
    Pseudocode:
        (x,y,theta) = cur_pose
        for i in range(18):
            d = getViews(x,y,theta)
            probArray[i] = gaussian(obs[i], d, sensor_sigma)
    """
    probArray = gaussian(obs[0], mapper.obs_views[:,:,:,0],sensor_sigma)
    for i in range(1,18): # Probability of getting ALL the sensor readings
                        # p(z1 & z2 & ...) = p(z1)p(z2)...
        probArray = probArray*gaussian(obs[i], mapper.obs_views[:,:,:,i],sensor_sigma)
        # this is elementwise multiplication
    # for debugging
#     eta = sum(sum(sum(probArray)))
#     print("In sensor model:")
#     if np.isfinite(eta):
#         print("Eta is finite and equals {}".format(eta))
#     elif abs(eta) < 1e-10:
#         print("Eta is zero!")
#     else:
#         print("Eta is infinite!")
#     print("probArray:")
#     print(probArray.shape)
    probArray = probArray / sum(sum(sum(probArray)))  # normalized probability
    # need to sum over all three indices to get total
    return probArray

# In configuration space
def update_step(bel_bar, obs):
    """ Update step of the Bayes Filter.
    Update the probabilities in loc.bel based on loc.bel_bar and the sensor model.
    Args:
        loc.bel_bar [ndarray 20x20x18]: belief after prediction step
        obs [ndarray]: A 1D array consisting of the measurements made in rotation loop
    Returns:
        loc.bel [ndarray 20x20x18]: belief after update step
    Pseudocode:
        loc.bel = sensorModel(obs)*loc.bel_bar
        eta = 1/sum(loc.bel)    # normalization constant
        loc.bel = loc.bel*eta
    """
    bel = sensor_model(obs)*bel_bar
    #bel = bel_bar  # for debugging: use only the motion model
    #bel = sensor_model(obs) # for debugging: use only the sensor model
    # for debugging
#     eta = sum(sum(sum(bel)))
#     print("In update step:")
#     if np.isfinite(eta):
#         print("Eta is finite and equals {}".format(eta))
#     elif abs(eta) < 1e-10:
#         print("Eta is zero!")
#     else:
#         print("Eta is infinite!")
#     print("bel:")
#     print(bel.shape)
    bel = bel/sum(sum(sum(bel))) # normalized
    # need to sum over all three indices to get total
    return bel

# Run the Bayes Filter
The cell below utilizes the functions defined above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In each iteration:
- Execute robot motion (get $u_{t}$ as previous and current odom) 
- Perform prediction step (calculate $\overline{bel}$)
- Print information regarding Prediction step
- Execute robot rotation behavior to get observation data (get $z_{t}$)
- Perform update step (calculate $bel$)
- Print information regarding update step


**<ins>NOTE</ins>**: 
- During initial testing, you may want to limit the iteration to only the first time step (i.e t=0) instead of looping through the entire trajectory.
- <span style="color:rgb(0,150,0)">If you make changes to any of the functions above, make sure to re-run the above cell before executing the cell below.</span>
- The functions *print_prediction_stats()* and *print_update_stats()* are helper functions defined in robot_interface.py and may be changed to suit your needs.

In [66]:
# Reset Plots
loc.plotter.reset_plot()

# Intialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
# for t in range(0,2): # testing: run with just a few steps

    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
    
    # Prediction Step
    tic = time.clock_gettime(time.CLOCK_MONOTONIC)
    loc.bel_bar = prediction_step(current_odom, prev_odom, loc.bel)
    loc.print_prediction_stats(plot_data=True)
    toc = time.clock_gettime(time.CLOCK_MONOTONIC)
    print("Prediction step time: {} s".format(toc-tic))
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    tic = time.clock_gettime(time.CLOCK_MONOTONIC)
    loc.bel = update_step(loc.bel_bar, loc.obs_range_data)
    loc.print_update_stats(plot_data=True)
    toc = time.clock_gettime(time.CLOCK_MONOTONIC)
    print("Update step time: {} s".format(toc-tic))
        
    print("-------------------------------------")



----------------- 0 -----------------
 | Resetting Robot pose

---------- PREDICTION STATS -----------
GT index            :  (11, 9, 7)
Prior Bel index     : (13,9,7) with prob = 0.8826
POS ERROR      : (-0.311, 0.005, 7.655)
---------- PREDICTION STATS -----------
Prediction step time: 10.862468661000094 s
 | Executing Observation Loop at: 30 deg/s

---------- UPDATE STATS -----------
GT index      :  (11, 9, 7)
Bel index     : (12,9,8) with prob = 1.0000
Bel_bar prob at index = 0.1943

GT     : (0.389, -0.095, -22.345)
Belief   : (0.500, -0.100, -10.000)
POS ERROR : (-0.111, 0.005, -12.345)
---------- UPDATE STATS -----------
Update step time: 0.030943489000492264 s
-------------------------------------


----------------- 1 -----------------

---------- PREDICTION STATS -----------
GT index            :  (13, 7, 8)
Prior Bel index     : (12,7,9) with prob = 0.8896
POS ERROR      : (0.172, 0.055, -15.157)
---------- PREDICTION STATS -----------
Prediction step time: 11.21093715200